In [1]:
from importlib import reload
import dataprep_functions as f

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [26]:
f = reload(f)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [2]:
census_df = f.merge_census(f.census_1, f.census_2)

In [4]:
# create datatables
#f.community_detection_iters()

In [5]:
# create a datatable
#f.calc_consen()

In [3]:
tract_df = f.create_tract_df_with_degree()

In [4]:
tract_df, community_df = f.create_community_df(tract_df)

In [5]:
community_df, com_mod_df = f.calc_community_modularity(community_df, tract_df)

/home/barcsab/urban_communities/scripts/dataprep_functions.py:329: RuntimeWarning: invalid value encountered in true_divide
  det = det / normadet


In [6]:
network_df = f.calc_modularity(com_mod_df)

In [7]:
community_df = f.calc_modularity_contribution(community_df, network_df)

In [8]:
community_df, network_df = f.calc_tract_number(tract_df, community_df, network_df, f.cbsacode)

In [9]:
network_df = f.calc_network_density(network_df)

In [10]:
network_df = f.calc_community_county_similarity(tract_df, network_df)

In [11]:
tract_df, census_df = f.clean_census_to_tract_df(tract_df, census_df)

In [12]:
community_df, network_df = f.calc_sum_and_std(tract_df, community_df, network_df)

In [13]:
tract_df, community_df = f.calc_income_percentile(tract_df, community_df)

In [14]:
tract_df = f.create_poor_dummy(tract_df)

In [27]:
tract_df, community_df = f.diff_from_network_avg(tract_df, community_df, network_df) # EZT KELL ÚJRA!!! 0615

In [ ]:
#tract_df.head(2).T

In [ ]:
#community_df.head(2).T

In [ ]:
#network_df.head(2).T

## Save tables

### Drop unnecessary columns

In [28]:
# county was calculated from geoid
tract_df = tract_df.drop(columns = ['ones', 'county', 'income_1_l'])

In [29]:
tract_df = tract_df.set_index('geoid')
community_df = community_df.set_index('S')
network_df = network_df.set_index('city')

In [30]:
tract_df.to_csv('../data/tract_level_data_2.csv.gz', compression = 'gzip')
community_df.to_csv('../data/community_level_data_2.csv.gz', compression = 'gzip')
#network_df.to_csv('../data/network_level_data.csv.gz', compression = 'gzip')